In [18]:
#Competition
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
from skimage import io
import matplotlib.pyplot as plt

def read_data():
    
    #get image filenames
    cat_locs = glob.glob('catsfolder/*.jpg')
    dog_locs = glob.glob('dogsfolder/*.jpg')
    num_cats = len(cat_locs)
    num_dogs = len(dog_locs)

    #initialize empty arrays
    X_cats = np.zeros((num_cats,64*64))
    X_dogs = np.zeros((num_dogs,64*64))
    y_cats = np.zeros((num_cats,1))
    y_dogs = np.zeros((num_dogs,1))
               
    #Load data, reshape into a 1D vector and set labels
    
    keep_track = 0

    for i in range(len(cat_locs)):
        img = cat_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_cats[i,:] = im
        y_cats[i] = -1.0
        keep_track += 1

    for i in range(len(dog_locs)):
        img = dog_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_dogs[i,:] = im
        y_dogs[i] = 1.0
        keep_track += 1
    
    # combine both datasets
    X = np.append(X_cats,X_dogs,0)
    y = np.append(y_cats,y_dogs)
    
    return X, y 

def split_data(X,y,testpercent):
        
    [n, d] = X.shape
    
    ntest = int(round(n*(float(testpercent)/100)))
    ntrain = int(round(n - ntest))
        
    Xtrain = np.zeros((ntrain,d))
    Xtest = np.zeros((ntest,d))
    ytrain = np.zeros((ntrain,1))
    ytest = np.zeros((ntest,1))   
        
    Data = np.column_stack((X,y))
    Data = np.random.permutation(Data)
    
    for i in range(ntest):
        Xtest[i,:] = Data[i,0:d]
        ytest[i] = Data[i,d]
        
    for i in range(ntrain):
        Xtrain[i,:] = Data[i+ntest,0:d]
        ytrain[i] = Data[i+ntest,d]
        
    return Xtrain, ytrain, Xtest, ytest

def show_image(X, i, label):
    #select image
    image = X[i,:]
    #reshape make into a square
    image = image.reshape((64,64))
    #display the image
    plt.title(label)
    plt.imshow(image,'gray')
    
def calculate_accuracy(ytrue, yguess):
    correct = sum(ytrue == yguess)
    total = len(ytrue)
    accuracy = 100*float(correct)/float(total)
    return accuracy

X,y = read_data()
Xtrain, ytrain, Xtest, ytest = split_data(X, y, 30)

class animalsDataset(Dataset):

    def __init__(self, X, y): 
        X_ = torch.tensor(X)
        self.X = torch.zeros(len(X), 1, 64, 64, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.int64)
        for k, i in enumerate(X_):
            new_i = i[np.newaxis, np.newaxis, :]
            new_i.resize_(1, 64, 64)
            self.X[k] = new_i
        self.X = self.X / 1000
        for k, i in enumerate(self.y):
            if i == -1:
                self.y[k] = 0
            elif i == 1:
                self.y[k] = 1
        self.y = self.y.clone().detach().squeeze()
            
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        sample = tuple((self.X[index], self.y[index]))

        return sample

In [8]:
trainData = animalsDataset(Xtrain, ytrain)
testData = animalsDataset(Xtest, ytest)

In [9]:
torch.manual_seed(1)

class Net(nn.Module):
    
    def __init__(self, Layers):
        super(Net, self).__init__()
        self.hidden = nn.ModuleList()
        for input_size, output_size in zip(Layers, Layers[1:]):
            self.hidden.append(nn.Linear(input_size, output_size))
    
    def forward(self, activation):
        L = len(self.hidden)
        for (l, linear_transform) in zip(range(L), self.hidden):
            if l < L - 1:
                activation = F.relu(linear_transform(activation))
            else:
                activation = linear_transform(activation)
                activation = F.softmax(activation, dim = 1)
        return activation
    
class CNN(nn.Module):
    
    def __init__(self, out_1=16, out_2=32):
        super(CNN, self).__init__()
        self.cnn0 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size = 4, stride=1, padding=1)
        self.conv0_bn = nn.BatchNorm2d(8)
        self.relu0 = nn.ReLU()
        self.maxpool0 = nn.MaxPool2d(kernel_size=2)
        self.cnn1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(1568, 2)
    
    # Prediction
    def forward(self, x):
        out = self.cnn0(x)
        out = self.conv0_bn(out)
        out = self.relu0(out)
        out = self.maxpool0(out)
        out = self.cnn1(out)
        out = self.conv1_bn(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.conv2_bn(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = F.softmax(out, dim = 1)
        return out
    
'''
model=nn.Sequential(
    nn.Linear(4096, 100),
    nn.ReLU(),
    nn.Linear(100, 2),
    nn.Softmax(dim = 1))
'''


#model = Net([4096, 1000, 1000, 100, 2])
model = CNN()

In [10]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=trainData, batch_size=10)
validation_loader = torch.utils.data.DataLoader(dataset=testData, batch_size=1)

In [11]:
n_epochs = 10
loss_list = []
accuracy_list = []
N_train = len(trainData)
N_validate = len(testData)

def train_model(n_epochs):
    for i, epoch in enumerate(range(n_epochs)):
        print("Epoch " + str(i + 1))
        correct = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            z = model(x) #z = model(x.view(-1, 64 * 64))
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y).sum().item()
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
        accuracy = correct / N_train
        print("Training accuracy: ", accuracy * 100, "%")

        
        correct=0  
        for x_test, y_test in validation_loader:
            z = model(x_test) #z = model(x_test.view(-1, 64 * 64))
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / N_validate
        accuracy_list.append(accuracy)
        loss_list.append(loss.data)
        print("Validation accuracy: ", round(accuracy * 100, 2), "%")
        print()
        
    return accuracy



In [12]:
accuracy = train_model(n_epochs)
print("Final accuracy: ", round(accuracy * 100, 2), "%")

Epoch 1
Training accuracy:  88.42857142857142 %
Validation accuracy:  92.0 %

Epoch 2
Training accuracy:  95.28571428571428 %
Validation accuracy:  94.83 %

Epoch 3
Training accuracy:  96.71428571428572 %
Validation accuracy:  96.17 %

Epoch 4
Training accuracy:  97.64285714285714 %
Validation accuracy:  96.0 %

Epoch 5
Training accuracy:  98.07142857142857 %
Validation accuracy:  96.67 %

Epoch 6
Training accuracy:  98.14285714285714 %
Validation accuracy:  97.5 %

Epoch 7
Training accuracy:  98.57142857142858 %
Validation accuracy:  97.83 %

Epoch 8
Training accuracy:  99.0 %
Validation accuracy:  98.0 %

Epoch 9
Training accuracy:  99.07142857142858 %
Validation accuracy:  97.83 %

Epoch 10
Training accuracy:  99.14285714285714 %
Validation accuracy:  97.83 %

Final accuracy:  97.83 %


In [25]:
#To run test on pre-trained model, run what is below. The only modification that must be made is changing the
#directories in read_data (or simply name the test directories as outlined in read_data()) so that X, y correspond 
#to the correct test data.

plt.close("all")
X, y = read_data()
Xtrain, ytrain, Xtest, ytest = split_data(X, y, 100)
testDataset = animalsDataset(Xtest, ytest)
final_loader = torch.utils.data.DataLoader(dataset=testDataset, batch_size=1)
yguess = np.zeros(len(ytest))
shown = 0
for i, (observation, label) in enumerate(final_loader):
    z = model(observation)
    _, yhat = torch.max(z.data, 1)
    yhat = yhat.item()
    if yhat == 0:
        guess = -1
        yguess[i] = guess
    elif yhat == 1:
        guess = 1
        yguess[i] = guess
    if(shown < 19 and guess != ytest[i]):
        label = "Cat: " + str(round(z[0][0].item() * 100)) + "% Dog: " + str(round(z[0][1].item() * 100)) + "%" 
        show_image(Xtest, i, label = label)
        plt.figure()
        shown += 1

ytest = np.reshape(ytest, 2000)

print("Accuracy: ", calculate_accuracy(ytest, yguess), "%")

Accuracy:  98.75 %
